In [1]:
#!pip uninstall numpy -y

In [2]:
#!pip uninstall transformers torch torchvision numpy -y

In [3]:
!pip install transformers torch torchvision numpy --upgrade

In [4]:
# Show versions:
!pip freeze | grep tensorflow
!pip freeze | grep torch
!pip freeze | grep numpy
# Show Python version
import os
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

tensorflow==2.18.0
tensorflow-datasets==4.9.8
tensorflow-hub==0.16.1
tensorflow-io-gcs-filesystem==0.37.1
tensorflow-metadata==1.17.1
tensorflow-probability==0.25.0
tensorflow-text==2.18.1
tensorflow_decision_forests==1.11.0
torch==2.7.0
torchao==0.10.0
torchaudio @ https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchdata==0.11.0
torchsummary==1.5.1
torchtune==0.6.1
torchvision==0.22.0
numpy==2.2.6
/usr/bin/python3
3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
sys.version_info(major=3, minor=11, micro=12, releaselevel='final', serial=0)


In [12]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Check PyTorch version
print(torch.__version__)
print(torch.cuda.is_available)

# torch.set_default_device(device='cuda:0')
# torch.get_default_device()

# print(torch.accelerator.current_accelerator().type)

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Define the text
texts = [
  "British writing from empire to Brexit : writing, identity and nation / Robert Spencer, Howard J. Booth, and Anastasia Valassopoulos. Abingdon, Oxon ; New York, NY : Routledge, Taylor & Francis Group, 2025. ©2025 National characteristics, British, in literature. European Union Great Britain. Great Britain In literature. Literature and society England History 19th century. Literature and society England History 20th century. English literature 19th century History and criticism. English literature 20th century History and criticism. Britanniques dans la littérature. (CaQQLa)000278323 Littérature et société (CaQQLa)201-0044931 Angleterre (CaQQLa)201-0476579 Histoire (CaQQLa)201-0378908 19e siècle. (CaQQLa)201-0378908 Littérature et société (CaQQLa)201-0044931 Angleterre (CaQQLa)201-0476579 Histoire (CaQQLa)201-0378892 20e siècle. (CaQQLa)201-0378892 Littérature anglaise (CaQQLa)201-0049945 19e siècle (CaQQLa)201-0049945 Histoire et critique. (CaQQLa)201-0377571 Littérature anglaise (CaQQLa)201-0021567 20e siècle (CaQQLa)201-0021567 Histoire et critique. (CaQQLa)201-0377571",
  "Contesting cosmopolitan moments in the long eighteenth century / Enit Karafili Steiner. Edinburgh : Edinburgh University Press, [2025] British literature 18th century History and criticism. Cosmopolitanism in literature.",
  "Radical writers and the media revolution in the late Enlightenment / Christoph Streb. Liverpool : Liverpool University Press, 2024. ©2024 British literature 18th century History. French literature 18th century History. Radicalism Great Britain 18th century History. Radicalism France 18th century History. Mass media Great Britain 18th century History. Mass media France 18th century History. Littérature britannique 18e siècle Histoire. Littérature française 18e siècle Histoire. Radicalisme Grande-Bretagne 18e siècle Histoire. Radicalisme France 18e siècle Histoire. Médias Grande-Bretagne 18e siècle Histoire. Médias France 18e siècle Histoire.",
  "A brief record of the Oliver government. [Vancouver] : Sun Job Presses, [1920] Campaign literature Canada. Campagnes électorales Canada Documentation. British Columbia Politics and government 1916-1928. Colombie-Britannique Politique et gouvernement 1916-1928. British Columbia Liberal Association. British Columbia Liberal Association.",
  "Misrule of the Oliver-Farris government. [Vancouver?] : Acme Press, [1920?] Campaign literature Canada. Campagne électorales Canada Documentation. British Columbia Politics and government 1916-1928. Colombie-Britannique Politique et gouvernement 1916-1928. Conservative Party of British Columbia. Conservative Party of British Columbia.",
  "To the electors of Yale : yours respectfully John P. McConnell. [British Columbia? : publisher not identified, 1912?] Campaign literature Canada. Campagnes électorales Documentation Canada. British Columbia Politics and government 1903-1916. Colombie-Britannique Politique et gouvernement 1903-1916. British Columbia Liberal Association. Liberal Party in British Columbia Platforms. British Columbia Liberal Association. Parti libéral de la Colombie-Britannique Programmes.",
  "Election address : Lieut.-Col. the Hon. E.G. Prior, Minister of Mines, the government candidate. [Victoria, B.C.? : publisher not identified, 1902?] Campaign literature British Columbia. Campagnes électorales Documentation Colombie-Britannique. British Columbia Politics and government 1871-1903. Colombie-Britannique Politique et gouvernement 1871-1903.",
  "The Business record of a business government. [Victoria, B.C.] : Colonist Presses, [1909?] Campaign literature, Canadian, 1916 Conservative Party of British Columbia. Campagnes électorales Canada Documentation, 1916 Conservative Party of British Columbia. British Columbia Politics and government 1903-1916. Colombie-Britannique Politique et gouvernement 1903-1916.",
  "A record of things done in behalf of the farmers : an account of what the Conservative government has done and is doing to promote the industry of agriculture in British Columbia and make easier the task of the tiller of the soil. [British Columbia? : publisher not identified, 1916?] Agricultural laws and legislation British Columbia. Agriculture and state British Columbia. Campaign literature, Canadian, 1916 Conservative Party of British Columbia. Campagnes électorales Canada Documentation, 1916 Conservative Party of British Columbia. Droit rural Colombie-Britannique. Politique agricole Colombie-Britannique.",
  "The platform of the Liberal Party of British Columbia. [Vancouver : Saturday Sunset Presses, 1916?] Campaign literature, Canadian, 1916 Liberal Party in British Columbia. Campagnes électorales Canada Documentation, 1916 Parti libéral de la Colombie-Britannique. British Columbia Politics and government 1903-1916. Colombie-Britannique Politique et gouvernement 1903-1916.",
  "Parry's third voyage for the discovery of a north-west passage in the years 1824 and 1825 : with an account of the Esquimaux / edited from Parry's first edition of his voyage with a biographical introduction. London; Glasgow : Blackie, 1894. Northwest Passage Juvenile literature. Inuit Juvenile literature. Inuit Ouvrages pour la jeunesse. Nord-Ouest, Passage du Ouvrages pour la jeunesse. Arctic regions Discovery and exploration British Juvenile literature. Régions arctiques Découverte et exploration britanniques Ouvrages pour la jeunesse. Parry, William Edward, Sir, 1790-1855 Juvenile literature. Parry, William Edward, Sir, 1790-1855 Ouvrages pour la jeunesse.",
  "The Apple lands of Aldergrove. New Westminster, B.C. : F.J. Hart and Company, [1910? (Vancouver] : Evans and Hastings) Agriculture British Columbia Aldergrove. Agriculture Colombie-Britannique Aldergrove. Canadian immigration literature (English) 1897-1918. Aldergrove (C.-B.) Descriptions et voyages. Aldergrove (B.C.) Description and travel. Canada Émigration et immigration Guides, manuels, etc. anglais 1897- 1918.",
  "Women of British Columbia : read the record of the Oliver government on matters in which you are vitally interested. Vancouver : Sun Job Presses, [1920?] Women Legal status, laws, etc. British Columbia. Campagnes électorales Canada Documentation, 1920 British Columbia Liberal Association. Femmes Droit Colombie-Britannique. Campaign literature, Canadian, 1920 British Columbia Liberal Association.",
  "Vancouver Island, British Columbia, Canada : the farmer' s home ; berry growing, vegetable raising, fruit growing, poultry raising, dairying, mixed farming, horses, cattle, sheep and hogs, good prices and steady markets ; the best all-year round mild climate in Canada, no blizzards, no six months winter sieges, no sunstrokes, no furnace heat summers ; there are many places for settlers but only one place ; and that is - Vancouver Island, B.C., Canada. [British Columbia? : publisher not identified, 1913?] Agriculture Colombie-Britannique Vancouver, ÃƒÂ®le de. Agriculture British Columbia Vancouver Island. Canadian immigration literature (English) 1897-1918. Canada Émigration et immigration Guides, manuels, etc. anglais 1897- 1918.6 Vancouver, ÃƒÂ®le de (C.-B.) Descriptions et voyages.0 Vancouver Island (B.C.) Description and travel.",
  "Discovering Britain and Ireland in the Romantic period : grand tours / edited by James Watt, Alison O'Byrne. Cambridge : Cambridge University Press, . Travelers' writings, British 19th century History and criticism. Travelers' writings, English Ireland History and criticism. Great Britain Description and travel. http://id.loc.gov/authorities/subjects/sh85056663 Ireland Description and travel. http://id.loc.gov/authorities/subjects/sh85067977 Romanticism Influence. Great Britain In literature. http://id.loc.gov/authorities/subjects/sh2007100286 Ireland In literature. http://id.loc.gov/authorities/subjects/sh2008104923",
  "William Morris and the aesthetic constitution of politics / Bradley J. Macdonald. Second edition. Lanham : Lexington Books, [2024] Morris, William, 1834-1896 Political and social views. Aesthetics Political aspects Great Britain History 19th century. Politics and literature Great Britain History 19th century. Art Political aspects Great Britain History 19th century. Great Britain Politics and government 1837-1901. Aesthetics, British 19th century. Politique et littérature (CaQQLa)201-0029043 Grande-Bretagne (CaQQLa)201-0441031 Histoire (CaQQLa)201-0378908 19e siècle. (CaQQLa)201-0378908 Grande-Bretagne Politique et gouvernement 1837-1901. (CaQQLa)201-0076985 Esthétique britannique 19e siècle. (CaQQLa)201-0132192",
  "Policy Document of the BC Conservative Party British Columbia's Conservatives. At head of title: Langley, B.C. : BC Conservative Party, 2012 Campaign literature British Columbia. Agriculture Iptcnc Alternative energy Iptcnc Bc hydro Iptcnc British columbia Iptcnc Government Iptcnc Health care Iptcnc Politics Iptcnc Supply management (canada) Iptcnc Tax Iptcnc Economy Iptcnc British Columbia Politics and government 2001-. British Columbia Conservative Party Platforms.",
  "Green book 2013 Victoria, B.C. : Green Party of BC, 2013 Campaign literature British Columbia. Agriculture Iptcnc Air pollution Iptcnc Disability Iptcnc Economy Iptcnc Education Iptcnc Employment Iptcnc Fishing Iptcnc Greenhouse gas Iptcnc Health Iptcnc Natural gas Iptcnc British Columbia Politics and government 2001-. Green Party of British Columbia Platforms. British Columbia. Legislative Assembly Elections, 2013.",
  "Strong Economy, Secure Tomorrow: 2013 Platform Victoria, B.C. : BC Liberal Party, 2013 Campaign literature British Columbia. British Columbia Politics and government. British Columbia Politics and government 2001- British Columbia Liberal Party Platforms. British Columbia. Legislative Assembly Elections, 2013.",
  "Families first agenda for change: Clark campaign policy proposals Clark campaign policy proposals. Vote for change. [Victoria, B.C.] : [BC Liberal Party], [2011] Family policy British Columbia. Campaign literature British Columbia. British columbia Iptcnc Crime Iptcnc Criminal justice Iptcnc Tax Iptcnc Venture capital Iptcnc Sustainable energy Iptcnc Government Iptcnc Smoking cessation Iptcnc Tourism Iptcnc Entrepreneurship Iptcnc British Columbia Politics and government. British Columbia Politics and government 2001- British Columbia Liberal Party Platforms. British Columbia. Premier Elections, 2011.",
  "Policies of the Christian Heritage Party of British Columbia Christian Heritage Party of British Columbia. Revised September 04, 2014. [Smithers, British Columbia] : Christian Heritage Party of British Columbia, [2014] Ottawa, Ontario : Canadian Electronic Library, 2015. Campaign literature British Columbia. Abortion Iptcnc British columbia Iptcnc Canada Iptcnc Citizenship Iptcnc Crime Iptcnc Election Iptcnc Electricity generation Iptcnc Federal government of the united states Iptcnc Government Iptcnc Health care Iptcnc British Columbia Politics and government 2001- British Columbia Politics and government. Christian Heritage Party of British Columbia Platforms.",
  "Return of the gods : mythology in romantic philosophy and literature / Owen Ware. First edition. New York, NY : Oxford University Press, [2024] ©2024 Mythology in literature. German literature 19th century History and criticism. English literature 19th century History and criticism. Romanticism Germany. Romanticism Great Britain. Philosophy, German 19th century. Philosophy, British 19th century.",
  "Harry Potter : a history of magic. History of magic London : Bloomsbury Publishing Plc, 2017. ©2017 Rowling, J. K. Harry Potter series History Exhibitions Juvenile literature. Rowling, J. K. Harry Potter series Miscellanea Exhibitions Juvenile literature. British Library Exhibitions Juvenile literature. Witchcraft History Exhibitions. Magic in literature Exhibitions. Wizards in literature Exhibitions. Rowling, J. K. Harry Potter series History Exhibitions. British Library Exhibitions. Harry Potter films. Magic History. Sorcellerie Histoire Expositions. Magie dans la littérature Expositions. Rowling, J. K. History. cct Rowling, J. K. cct British Library. cct Harry Potter films Juvenile literature. cct Magic History Juvenile literature. cct Harry Potter films. cct Magic History. cct Magic. cct Exhibition catalogs. cct History. cct Juvenile works. cct Literature. eflch British Library fast Harry Potter series (Rowling, J. K.) fast Wizards in literature fast Witchcraft fast Magic in literature fast Harry Potter films fast Magic fast Rowling, J. K. 1965- Harry Potter gnd Harry Potter films. sears Literature. ukslc exhibition catalogs. aat Exhibition catalogs fast History fast Juvenile works fast Exhibition catalogs. lcgft Catalogues d'exposition. rvmgf",
  "Masculinities and representation : the eroticized male in early modern Italy and England / edited by Konrad Eisenbichler. Toronto ; Buffalo ; London : University of Toronto Press, [2024] Masculinity Italy History Early modern, 1500-1700. Masculinity England History Early modern, 1500-1700. Masculinity in literature. Masculinity in art. Arts, Italian History Early modern, 1500-1700. Arts, British History Early modern, 1500-1700. Masculinity Social aspects History Early modern, 1500-1700. Masculinity Italy History. Masculinity England History. Masculinité Italie Histoire. Masculinité Angleterre Histoire. Masculinité dans la littérature. Masculinité dans l'art.",
  "British book news. No. 69 : a monthly selection of recent books. London : National Book League, 1946. Blind Books and reading. http://id.loc.gov/authorities/subjects/sh85014856 British literature. http://id.loc.gov/authorities/subjects/sh85016973",
  "Review 43, a quarterly review of literature, art and science / edited by Walter Berger and Pavel Tigrid. London : George Allen and Unwin, Limited, 1943. Art, British 20th century. http://id.loc.gov/authorities/subjects/sh2003010015 Art, Czech 20th century. http://id.loc.gov/authorities/subjects/sh2006002697 Czech literature 20th century. http://id.loc.gov/authorities/subjects/sh85035277 Slovak literature 20th century. http://id.loc.gov/authorities/subjects/sh85123481",
  "Art and the industrial revolution / by Francis D. Klingender. London : Noel Carrington, 1947. Art, British. http://id.loc.gov/authorities/subjects/sh85007581 English literature 18th century History and criticism. http://id.loc.gov/authorities/subjects/sh2008102755 English literature 19th century History and criticism. http://id.loc.gov/authorities/subjects/sh2008102754 Industries in art. http://id.loc.gov/authorities/subjects/sh85065979 Industries in literature. http://id.loc.gov/authorities/subjects/sh85065980",
  "A short essay on the subject of Negro slavery, with a particular reference to the island of Barbadoes. By the Reverend H.E. Holder, of that place [electronic resource]. London : Printed by Couchman and Fry, for Charles Dilly, in the Poultry, MDCCLXXXVIII. [1788] Slavery West Indies, British Controversial literature.",
  "The European magazine, and London review [electronic resource] : containing the literature, history, politics, arts, manners and amusements of the age. By the Philological Society of London. European magazine, and London review London [England] : printed for John Fielding No. 23, Pater Noster Row; John Debrett, opposite Burlington House, Piccadilly; and John Sewell, No. 32, Cornhill, [1782]-1826. British literature History 18th century Periodicals Early works to 1800. Great Britain Civilization 18th century Periodicals Early works to 1800. Great Britain Social life and customs 18th century Periodicals Early works to 1800. Periodicals England 18th century. rbgenr Essays Great Britain 18th century. rbgenr",
  "Explorers, dreamers and thieves : writers in the archives in the British Museum / Selva Almada, Rita Indiana, Josefa Sánchez Contreras, Philippe Sands, Juan Gabriel Vásquez, Gabriela Wiener ; translated by Anne McLean, Robin Myers [and three others]. Edinburgh : Charco Press, 2024. British Museum Essays. Museum objects Poetry. Latin American literature. Objets exposés Poésie. Littérature latino-américaine.",
  "Sylvia Beach and the lost generation Sylvia Beach : Eine Biographie im literarischen Paris 1920-1940 / Noel R. Fitch ; Aus dem Amerikanischen von Angelika Schleindl Insel. Nordlingen : Wagner, 1988. Beach, Sylvia. Booksellers and bookselling France Paris Biography. Paris (France) Intellectual life 20th century. Americans France Paris History 20th century. British France Paris History 20th century. Literature publishing France Paris History 20th century. Literary landmarks France Paris History 20th century. Joyce, James, 1882-1941. Beach Sylvia Biographie 1920-1940. Paris (France) Vie intellectuelle 20e siècle. Britanniques France Paris Histoire 20e siècle. Littérature Édition France Paris Histoire 20e siècle. Écrivains Résidences et lieux familiers France Paris Histoire 20e siècle. Beach, Sylvia fast (OCoLC)fst00044498 https://id.oclc.org/worldcat/entity/E39PBJgH7gjGjx7dpV8trt48YP Joyce, James, 1882-1941 fast (OCoLC)fst00035968 https://id.oclc.org/worldcat/entity/E39PBJcGrYrqpkRpQrMfc4bjmd Americans. fast (OCoLC)fst00807488 Booksellers and bookselling. fast (OCoLC)fst00836492 British. fast (OCoLC)fst00839044 Intellectual life. fast (OCoLC)fst00975769 Literary landmarks. fast (OCoLC)fst00999933 Literature publishing. fast (OCoLC)fst01000111 France Paris fast (OCoLC)fst01205283 https://id.oclc.org/worldcat/entity/E39QbtfRkKTXcHQJ477vgbjTxc Beach, Sylvia. swd 1900-1999 fast Biographies. fast (OCoLC)fst01919896 History. fast (OCoLC)fst01411628",
  "Pity the negro, or, An address to children on the subject of slavery. Address to children on the subject of slavery 7th ed. London : F. Westley and A.H. Davis, 1827. Slavery West Indies, British Juvenile literature. Slave trade Great Britain Juvenile literature. Slaves Emancipation Juvenile literature.",
  "The history of the black slaves : being an affectionate address to the children of Great Britain, of all ranks and denominations / by an old friend to the young. Leicester : Printed and sold by A. Cockshaw ..., [1830?] Slavery Moral and ethical aspects Juvenile literature. Slavery West Indies, British Juvenile literature. Slave trade Juvenile literature. Antislavery movements Great Britain Juvenile literature.",
  "The inhuman empire : wildlife, colonialism, culture / Sadhana Naithani. New York : Routledge, Taylor & Francis Group, 2024. ©2024 Animals in literature. Hunting in literature. India In literature. English literature 19th century History and criticism. English literature 20th century History and criticism. British India History. India History British occupation, 1765-1947. Great Britain Colonies Asia. Animaux dans la littérature. (CaQQLa)201-0001369 Littérature anglaise (CaQQLa)201-0049945 19e siècle (CaQQLa)201-0049945 Histoire et critique. (CaQQLa)201-0377571 Littérature anglaise (CaQQLa)201-0021567 20e siècle (CaQQLa)201-0021567 Histoire et critique. (CaQQLa)201-0377571 Britanniques (CaQQLa)201-0003276 Inde (CaQQLa)201-0482676 Histoire. (CaQQLa)201-0378888 Inde Histoire 1765-1947 (Occupation britannique) (CaQQLa)201-0013890 Grande-Bretagne Colonies Asie. (CaQQLa)201-0163246",
  "Black travel writing : contemporary narratives of travel to Africa by African American and Black British authors / Isabel Kalous. 1st ed. Bielefeld : transcript Verlag, [2021] ©2021 American literature History and criticism. British literature History and criticism.",
  "Consuming anxieties : alcohol, tobacco, and trade in British satire, 1660-1751 / Dayne C. Riley. Alcohol, tobacco, and trade in British satire, 1660-1751 Lewisburg, Pennsylvania : Bucknell University Press, [2024] English literature 17th century History and criticism. English literature 18th century History and criticism. Drinking of alcoholic beverages in literature. Tobacco in literature. Satire, English History and criticism. Literature and society Great Britain History 17th century. Literature and society Great Britain History 18th century. National characteristics, British, in literature. Literary criticism. lcgft",
  "Empire of culture : neo-Victorian narratives in the global creative economy / Waiyee Loh. Albany : State University of New York Press, 2024. English literature 19th century History and criticism. Great Britain Colonies History. Civilization British influences. British Foreign countries. Littérature anglaise (CaQQLa)201-0049945 19e siècle (CaQQLa)201-0049945 Histoire et critique. (CaQQLa)201-0377571 Grande-Bretagne Colonies Histoire. (CaQQLa)201-0163217 Britanniques Pays étrangers. (CaQQLa)201-0106047",
  "Critical perspectives on resistance in 21-st century British literature / edited by Nilay Erdem Ayyildiz. Newcastle upon Tyne : Cambridge Scholars Publishing, 2024. ©2024 British literature History and criticism 21st century. Resistance (Philosophy) in literature.",
  "Out of hiding : Holocaust literature of British Columbia : an historical survey in appreciation of Robert Krell / Alan Twigg ; afterword by Yosef Wosk. Vancouver, B.C., Canada : Ronsdale Press, [2022] ©2022 Canadian literature British Columbia History and criticism. Holocaust survivors' writings. Holocaust survivors' writings History and criticism. Holocaust, Jewish (1939-1945), in literature. Holocaust survivors British Columbia. Jews British Columbia. Jews, Canadian British Columbia. Canadian literature. fast (OCoLC)fst00845111 Holocaust, Jewish (1939-1945) in literature. fast (OCoLC)fst00958923 Holocaust survivors. fast (OCoLC)fst00958838 Holocaust survivors' writings. fast (OCoLC)fst00958863 Jews. fast (OCoLC)fst00983135 Jews, Canadian. fast (OCoLC)fst00983427 British Columbia. fast (OCoLC)fst01205265 Canadian literature British Columbia History and criticism nli Holocaust survivors' writings nli Holocaust survivors' writings History and criticism nli Holocaust, Jewish (1939-1945), in literature nli Holocaust survivors British Columbia nli Jews British Columbia nli Literary criticism fast (OCoLC)fst01986215 Criticism, interpretation, etc. fast (OCoLC)fst01411635 Literary criticism. lcgft",
  "Finnegans Wakes : Tales of Translation / Patrick O'Neill. First edition. Toronto, Ontario : University of Toronto Press, [2022] ©2022 British literature. Joyce, James, 1882-1941 Translations History and criticism. Joyce, James, 1882-1941. Finnegans wake.",
  "The Savage and Modern Self : North American Indians in Eighteenth-Century British Literature and Culture / Robbie Richardson. Toronto : University of Toronto Press, [2019] ©2018 Indians in literature. English literature 18th century History and criticism. National characteristics, British, in literature."
]

def get_sentence_embedding(text):
    # Tokenize and prepare input tensors
    inputs = tokenizer(text, return_tensors='pt')
    # Disable gradient calculation
    with torch.no_grad():
        # Get model outputs
        outputs = model(**inputs)
    # Extract the last hidden states (embeddings)
    last_hidden_states = outputs.last_hidden_state
    # Average token embeddings
    sentence_embedding = torch.mean(last_hidden_states, dim=1).numpy()
    return sentence_embedding

# Generate embeddings for texts
embeddings = [get_sentence_embedding(text) for text in texts]

# Query text
query_text = "british literature"
query_embedding = get_sentence_embedding(query_text)

# Compute cosine similarities
# Lokk for another similarity index that I could use.
similarities = cosine_similarity(query_embedding, np.vstack(embeddings))

# Print query text
print (f"Query text: {query_text}")

# Print similarities
for i, text in enumerate(texts):
    print(f"Similarity with '{text}': {similarities[0][i]}")


2.7.0+cu126
<function is_available at 0x786de6141b20>
Query text: british literature
Similarity with 'British writing from empire to Brexit : writing, identity and nation / Robert Spencer, Howard J. Booth, and Anastasia Valassopoulos. Abingdon, Oxon ; New York, NY : Routledge, Taylor & Francis Group, 2025. ©2025 National characteristics, British, in literature. European Union Great Britain. Great Britain In literature. Literature and society England History 19th century. Literature and society England History 20th century. English literature 19th century History and criticism. English literature 20th century History and criticism. Britanniques dans la littérature. (CaQQLa)000278323 Littérature et société (CaQQLa)201-0044931 Angleterre (CaQQLa)201-0476579 Histoire (CaQQLa)201-0378908 19e siècle. (CaQQLa)201-0378908 Littérature et société (CaQQLa)201-0044931 Angleterre (CaQQLa)201-0476579 Histoire (CaQQLa)201-0378892 20e siècle. (CaQQLa)201-0378892 Littérature anglaise (CaQQLa)